In [1]:
import os
import re
import pendulum as pendulum
from pymongo import MongoClient
import numpy as np
import pandas as pd
import time
from datetime import datetime,timedelta, timezone
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

pd.set_option('mode.chained_assignment',  None)

In [6]:
# 음성인식 완료 노트수

# df_quota 폴더 내의 파일 목록 가져오기
quota_folder = '../raw_data/df_quota'
quota_files = sorted([file for file in os.listdir(quota_folder) if file.endswith('.csv')])

# df_recognition 폴더 내의 파일 목록 가져오기
recognition_folder = '../raw_data/df_recognition'
recognition_files = sorted([file for file in os.listdir(recognition_folder) if file.endswith('.csv')])

# df_uploadtype 폴더 내의 파일 목록 가져오기
uploadtype_folder = '../raw_data/df_uploadtype'
uploadtype_files = sorted([file for file in os.listdir(uploadtype_folder) if file.endswith('.csv')])

# df_quota_byuploadtype 폴더 생성
output_folder = '../raw_data/df_quota_byuploadtype'
os.makedirs(output_folder, exist_ok=True)


# 각 월별로 동일한 월의 두 데이터프레임 합치기
for uploadtype_file in uploadtype_files:
    year_month = re.search(r'df_uploadtype_(\d{4}_\d{2})\.csv', uploadtype_file).group(1)
    
    # df_quota 폴더에 동일한 월의 파일 찾기
    quota_file = f"df_quota_{year_month}.csv"
    if quota_file in quota_files:
        # 해당 월의 df_quota와 df_uploadtype 데이터프레임 읽어오기
        uploadtype_file_path = os.path.join(uploadtype_folder, uploadtype_file)
        quota_file_path = os.path.join(quota_folder, quota_file)
        df_uploadtype = pd.read_csv(uploadtype_file_path)
        df_quota = pd.read_csv(quota_file_path)
        
        # 해당 월의 df_recognition 파일 경로 설정
        if year_month >= '2023_11':
            recognition_file = f"df_recognition_{year_month}.csv"
            recognition_file_path = os.path.join(recognition_folder, recognition_file)
            # df_recognition 데이터프레임 읽어오기
            df_recognition = pd.read_csv(recognition_file_path)
            
            # df_quota와 df_recognition을 concat하기
            concat_df = pd.concat([df_quota, df_recognition], ignore_index=True)
        else:
            # df_recognition가 없는 경우, df_quota만 사용
            concat_df = df_quota
            
        # df_uploadtype과 조인하기
        merged_df = pd.merge(df_uploadtype, concat_df, on=['userId', 'noteId'], how='inner')
        
        # 중복된 'noteId' 제거
        merged_df = merged_df.drop_duplicates(subset='noteId', keep='first')
        
        output_file_path = os.path.join(output_folder, f'df_quota_byuploadtype_{year_month}.csv')
        merged_df.to_csv(output_file_path, index=False)

/tmp/ipykernel_61659/2215856318.py:30: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_uploadtype = pd.read_csv(uploadtype_file_path)
/tmp/ipykernel_61659/2215856318.py:31: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_quota = pd.read_csv(quota_file_path)


AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
folder_path = '../raw_data/df_quota_byuploadtype'
file_names = sorted([file for file in os.listdir(folder_path) if file.endswith('.csv')])

# 작업을 수행하고 CSV 파일로 저장
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    
    # 파일 읽기
    df = pd.read_csv(file_path)
    
    # uploadType 매핑
    uploadType_mapping = {
        'CLOUD_RECORDING': 'ZOOM',
        'LIVE_STREAMING': 'ZOOM',
        'RECORDING': 'RECORD'
    }
    df['uploadtype'] = df['value_x'].replace(uploadType_mapping)

    # clientType 매핑
    clientType_mapping = {
        '1001': 'WEB',
        '2101': 'ANDROID',
        '2201': 'IOS',
        '3011': 'ZOOM'
    }
    df['clientType'] = df['clientType_x'].astype(str).replace(clientType_mapping)
    # 불필요한 열 삭제
    df.drop(columns=['value_x', 'clientType_x'], inplace=True)

    # 수정된 데이터프레임을 다시 CSV 파일로 저장
    df.to_csv(file_path, index=False)

In [ ]:
monthly_results = {}

# df_quota_byuploadtype 폴더 내의 파일 목록 가져오기
folder_path = '../raw_data/df_quota_byuploadtype'
file_names = sorted([file for file in os.listdir(folder_path) if file.endswith('.csv')])

# 각 파일에 대해 작업 수행
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    
    df = pd.read_csv(file_path)
      
    year_month = re.search(r'df_quota_byuploadtype_(\d{4}_\d{2})\.csv', file_name).group(1)

    # clientType와 uploadType 별로 그룹화하여 unique noteId 수와 sum(value_y) 계산
    grouped_data = df.groupby(['clientType', 'uploadtype']).agg({'userId': 'nunique', 'noteId': 'nunique', 'value_y': 'sum'}).reset_index()

    # Ensure 'value_y' column is numeric
    # grouped_data['value_y'] = pd.to_numeric(grouped_data['value_y'], errors='coerce')

    # value_y 값을 sum하고 3600으로 나눈 값을 'quota'열로 추가
    grouped_data['quota'] = grouped_data['value_y'] / 3600
    
    # 연도와 월별로 결과 저장
    monthly_results.setdefault(year_month, []).append(grouped_data)
    
    
# 각 월별 결과를 CSV 파일로 저장
for year_month, data in monthly_results.items():
    final_result = pd.concat(data)

    # 결과를 CSV 파일로 저장 (연도와 월을 포함하여 파일명 생성)
    output_file_path = os.path.join('../raw_data/monthly_results', f'quota_bygroup_{year_month}.csv')

    # 'value_y' 열 삭제
    final_result.drop(columns=['value_y'], inplace=True)  
    final_result.to_csv(output_file_path, index=False) 

In [ ]:
# raw_data/monthly_results 폴더 내의 CSV 파일 목록 가져오기
folder_path = '../raw_data/monthly_results'
file_names = sorted([file for file in os.listdir(folder_path) if file.endswith('.csv')])

monthly_dataframes = []

for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)

    df = pd.read_csv(file_path)

    year_month = re.search(r'quota_bygroup_(\d{4}_\d{2})\.csv', file_name).group(1)

    # 각 clientType와 uploadtype의 noteId 합계 계산
    total_note = df.groupby(['clientType', 'uploadtype'])['noteId'].sum().reset_index()

    total_note['year_month'] = year_month
    
    # 각 clientType와 uploadtype 별로 컬럼명 생성하여 noteId 수 저장
    for index, row in total_note.iterrows():
        column_name = f"{row['clientType']}-{row['uploadtype']}"
        total_note.at[index, column_name] = row['noteId']

    total_note.fillna(0, inplace=True)

    # 필요없는 clientType와 uploadtype 열 삭제
    total_note.drop(columns=['clientType', 'uploadtype', 'noteId'], inplace=True)

    monthly_dataframes.append(total_note)

note_bygroup = pd.concat(monthly_dataframes, ignore_index=True)
note_bygroup = note_bygroup.groupby('year_month').sum().reset_index()

print(note_bygroup)

In [ ]:
note_bygroup

In [ ]:
x_values = note_bygroup['year_month']
android_file = note_bygroup['ANDROID-FILE']
android_record = note_bygroup['ANDROID-RECORD']
ios_file = note_bygroup['IOS-FILE']
ios_record = note_bygroup['IOS-RECORD']
web_file = note_bygroup['WEB-FILE']
web_stream = note_bygroup['WEB-STREAM']
zoom_zoom = note_bygroup['ZOOM-ZOOM']

# colors = ['#1f77b4', '#2ca02c', '#ff7f0e', '#d62728', '#9467bd', '#8c564b', '#e377c2']
colors = ['#1f77b4', '#aec7e8', '#2ca02c', '#98df8a', '#9467bd', '#c5b0d5', '#ff7f0e']

plt.figure(figsize=(10, 6))

plt.bar(x_values, android_file, label='ANDROID-FILE', color=colors[0], alpha=0.7)
plt.bar(x_values, android_record, bottom=android_file, label='ANDROID-RECORD', color=colors[1], alpha=0.7)
plt.bar(x_values, ios_file, bottom=android_file+android_record, label='IOS-FILE', color=colors[2], alpha=0.7)
plt.bar(x_values, ios_record, bottom=android_file+android_record+ios_file, label='IOS-RECORD', color=colors[3], alpha=0.7)
plt.bar(x_values, web_file, bottom=android_file+android_record+ios_file+ios_record, label='WEB-FILE', color=colors[4], alpha=0.7)
plt.bar(x_values, web_stream, bottom=android_file+android_record+ios_file+ios_record+web_file, label='WEB-STREAM', color=colors[5], alpha=0.7)
plt.bar(x_values, zoom_zoom, bottom=android_file+android_record+ios_file+ios_record+web_file, label='ZOOM-ZOOM', color=colors[6], alpha=0.7)


# 토탈 노트수 표시
total_notes = note_bygroup.sum(axis=1)
for i, note_count in enumerate(total_notes):
    plt.text(i, note_count, f'{int(note_count)/1e6:.1f}M', ha='center', va='bottom')

# 그래프에 제목과 축 레이블 추가
plt.title('Monthly Created Note', fontsize = 16, fontweight = 'bold', pad = 20)
plt.xlabel('Month', fontsize = 14, fontweight = 'bold')
plt.ylabel('Note Count', fontsize = 14, fontweight = 'bold')

# y 축 숫자 표시 형식 변경 (정수 형식으로)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))

# 범례 표시
plt.legend()

# 그래프 출력
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# plt.savefig('graph/4. new notes.jpg', dpi=100, bbox_inches='tight')

In [ ]:
# 쿼터사용량

In [ ]:
folder_path = '../raw_data/monthly_results'
file_names = sorted([file for file in os.listdir(folder_path) if file.endswith('.csv')])

monthly_dataframes = []

for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)

    df = pd.read_csv(file_path)

    year_month = re.search(r'quota_bygroup_(\d{4}_\d{2})\.csv', file_name).group(1)

    # 각 clientType와 uploadtype의 noteId 합계 계산
    total_quota = df.groupby(['clientType', 'uploadtype'])['quota'].sum().reset_index()

    total_quota['year_month'] = year_month
    
    # 각 clientType와 uploadtype 별로 컬럼명 생성하여 noteId 수 저장
    for index, row in total_quota.iterrows():
        column_name = f"{row['clientType']}-{row['uploadtype']}"
        total_quota.at[index, column_name] = row['quota']

    # 필요없는 clientType와 uploadtype 열 삭제
    total_quota.drop(columns=['clientType', 'uploadtype', 'quota'], inplace=True)

    # 각 월별 데이터프레임을 리스트에 추가
    monthly_dataframes.append(total_quota)

quota_bygroup = pd.concat(monthly_dataframes, ignore_index=True)
quota_bygroup = quota_bygroup.groupby('year_month').sum().reset_index()

In [ ]:
quota_bygroup

In [ ]:
plt.figure(figsize=(10, 5))

x_values = quota_bygroup['year_month']
android_file = quota_bygroup['ANDROID-FILE']
android_record = quota_bygroup['ANDROID-RECORD']
ios_file = quota_bygroup['IOS-FILE']
ios_record = quota_bygroup['IOS-RECORD']
web_file = quota_bygroup['WEB-FILE']
web_stream = quota_bygroup['WEB-STREAM']
zoom_zoom = quota_bygroup['ZOOM-ZOOM']


colors = ['#1f77b4', '#aec7e8', '#2ca02c', '#98df8a', '#9467bd', '#c5b0d5', '#ff7f0e']

plt.figure(figsize=(10, 6))
plt.bar(x_values, android_file, label='ANDROID-FILE', color=colors[0], alpha=0.7)
plt.bar(x_values, android_record, bottom=android_file, label='ANDROID-RECORD', color=colors[1], alpha=0.7)
plt.bar(x_values, ios_file, bottom=android_file+android_record, label='IOS-FILE', color=colors[2], alpha=0.7)
plt.bar(x_values, ios_record, bottom=android_file+android_record+ios_file, label='IOS-RECORD', color=colors[3], alpha=0.7)
plt.bar(x_values, web_file, bottom=android_file+android_record+ios_file+ios_record, label='WEB-FILE', color=colors[4], alpha=0.7)
plt.bar(x_values, web_stream, bottom=android_file+android_record+ios_file+ios_record+web_file, label='WEB-STREAM', color=colors[5], alpha=0.7)
plt.bar(x_values, zoom_zoom, bottom=android_file+android_record+ios_file+ios_record+web_file, label='ZOOM-ZOOM', color=colors[6], alpha=0.7)


# 토탈 노트수 표시
total_quota = quota_bygroup.sum(axis=1)
for i, quota in enumerate(total_quota):
    plt.text(i, quota, f'{int(quota)/1e6:.1f}M', ha='center', va='bottom')

# 그래프에 제목과 축 레이블 추가
plt.title('Monthly Used Quota', fontsize = 16, fontweight = 'bold', pad = 20)
plt.xlabel('Month', fontsize = 14, fontweight = 'bold')
plt.ylabel('Quota-Hours', fontsize = 14, fontweight = 'bold')

# y 축 숫자 표시 형식 변경 (정수 형식으로)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))

# 범례 표시
plt.legend()

# 그래프 출력
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# plt.savefig('graph/5. monthly_quota.png', dpi=300, bbox_inches='tight')

In [ ]:
# 중복 row test

In [2]:
df_q_u_2024_02 = pd.read_csv('../raw_data/df_quota_byuploadtype/df_quota_byuploadtype_2024_02.csv')

In [3]:
len(df_q_u_2024_02)

1431367

In [4]:
df_q_u_2024_02['noteId'].nunique()

1431367

In [5]:
df_q_u_2024_02[df_q_u_2024_02['noteId']=='a270a6b7-1f3d-4ee1-803c-2a9152f602a7n']

,userId,noteId,category_x,action_x,createdDate_x,clientType_y,category_y,action_y,value_y,createdDate_y,uploadtype,clientType
77180,8c3cb631-8c5d-4501-be08-9d0d98c0ee67u,a270a6b7-1f3d-4ee1-803c-2a9152f602a7n,note,note.uploadType,2024-02-02 01:20:44.279,IOS,quota,quota.used,2860,2024-02-20 14:30:28.759,RECORD,IOS
